<a href="https://colab.research.google.com/github/SejinHan25/crack_segmentation/blob/main/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Load

In [ ]:
!unzip -qq "/content/drive/MyDrive/Colab Notebooks/crack_segmentation/traincrop.zip"

## 1. 모듈 및 데이터셋 불러오기

In [32]:
# !pip install segmentation-models-pytorch
# !pip install pytorch-lightning
# !pip install torchtext
# !pip install adamp

In [ ]:
import os 
import cv2
import torch
import torchvision
import PIL

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
import torch.optim.lr_scheduler as lr_scheduler

from adamp import AdamP
from torchmetrics.functional import jaccard_index
from glob import glob
from torchvision import transforms as T
from torch.utils.data import DataLoader, Dataset
from tqdm.autonotebook import tqdm
from sklearn.model_selection import train_test_split
from ipywidgets import interact
from torchvision.transforms.functional import to_pil_image
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# from save import save_model

In [ ]:
data_dir = "/content/traincrop/"
images_paths = glob(data_dir + "*.jpg")
masks_paths = glob(data_dir + "*.png")

images_paths = sorted([str(p) for p in images_paths])
masks_paths = sorted([str(p) for p in masks_paths])

df = pd.DataFrame({'images': images_paths, 'masks': masks_paths})
df.head(5)

,images,masks
0,/content/traincrop/20160222_081011_1281_721.jpg,/content/traincrop/20160222_081011_1281_721.png
1,/content/traincrop/20160222_081011_1921_721.jpg,/content/traincrop/20160222_081011_1921_721.png
2,/content/traincrop/20160222_081011_1_361.jpg,/content/traincrop/20160222_081011_1_361.png
3,/content/traincrop/20160222_081011_1_721.jpg,/content/traincrop/20160222_081011_1_721.png
4,/content/traincrop/20160222_081011_641_361.jpg,/content/traincrop/20160222_081011_641_361.png


In [ ]:
train, valid = train_test_split(df, test_size=0.1, shuffle=True, random_state=42)

print(f"Train size: {len(train)}, Validation size: {len(valid)}")

Train size: 1656, Validation size: 184


In [ ]:
@interact(index=(0, len(df)-1))
def show_images(index=0):
    image = cv2.imread(df.iloc[index].images)
    mask = cv2.imread(df.iloc[index].masks)

    plt.figure(figsize=(12,10))
    plt.subplot(121)
    plt.title("image")
    plt.imshow(image)
    plt.subplot(122)
    plt.title("mask")
    plt.imshow(mask)
    plt.tight_layout()

interactive(children=(IntSlider(value=0, description='index', max=1839), Output()), _dom_classes=('widget-inte…

## 2. 데이터셋 구축

### Albumentations

In [ ]:
import random
import albumentations as A

from albumentations.pytorch.transforms import ToTensorV2

In [ ]:
train_transform = [
    A.Resize(384, 640),
    A.RandomRain(brightness_coefficient=0.9, drop_width=1, blur_value=3, p=1),
    # A.RandomShadow(num_shadows_lower=1, num_shadows_upper=1, shadow_dimension=5, shadow_roi=(0, 0.5, 1, 1), p=1),
    # A.RandomSnow(brightness_coeff=1, snow_point_lower=0.1, snow_point_upper=0.3, p=1),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, brightness_by_max=True, p=1),
    A.CLAHE(clip_limit=2, tile_grid_size=(8, 8), p=1),
    ToTensorV2()
    ]

train_transform = A.Compose(train_transform)

In [ ]:
class SegmentationDataset(torch.utils.data.Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.loc[index].squeeze()
        image_path = row['images']
        mask_path = row['masks']
        
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, 0) // 255
        
        if self.transform:
            transformed = self.transform(image=image, mask=mask)
            image = transformed['image'].float()
            mask = transformed['mask'].float()
        
        return image, mask

In [ ]:
train_ds = SegmentationDataset(df, train_transform)

In [ ]:
train_ds[0][0].shape

torch.Size([3, 384, 640])

In [ ]:
@interact(index=(0, len(df)-1))
def show_images(index=0):
    plt.figure(figsize=(20,10))
    plt.axis=("off")
    plt.subplot(121)
    plt.imshow(train_ds[index][0].cpu().detach().numpy().transpose(1,2,0))
    plt.subplot(122)
    plt.imshow(np.squeeze(train_ds[index][1]).cpu().detach().numpy(), cmap='gray')

interactive(children=(IntSlider(value=0, description='index', max=1839), Output()), _dom_classes=('widget-inte…

In [ ]:
class SegmentationModel(pl.LightningModule):
    def __init__(self, args=None, optimizer='adam', scheduler='reducelr'):
        super().__init__()
        self.model = smp.UnetPlusPlus(
            encoder_name='efficientnet-b0',  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
            encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
            in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
            classes=2,  # model output channels (number of classes in your dataset)
        )
        self.args = args
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optimizer
        self.scheduler = scheduler

    def forward(self, x):
        x = self.model(x)
        return x

    def configure_optimizers(self):
        if self.optimizer == 'adam':
            optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        # elif self.args.optimizer == 'adamw':
        #     optimizer = torch.optim.AdamW(self.parameters(), lr=0.001)
        # elif self.args.optimizer == 'adamp':
        #     optimizer = AdamP(self.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-2)

        if self.scheduler == "reducelr":
            scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5, mode="max", verbose=True)
            return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val/jaccard_index_value"}

        # elif self.args.scheduler == "cosineanneal":
        #     scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=1e-5,
        #                                                          last_epoch=-1, verbose=True)

        return {"optimizer": optimizer, "lr_scheduler": scheduler}

    def training_step(self, train_batch, batch_idx):
        image, mask = train_batch
        mask = mask.long()

        outputs = self.model(image)
        loss = self.criterion(outputs, mask)
        jaccard_index_value = jaccard_index(outputs.argmax(dim=1), mask, num_classes=2)

        self.log('train/loss', loss, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log('train/jaccard_index_value', jaccard_index_value, on_epoch=True, on_step=True, prog_bar=True,
                 sync_dist=True)

        return {"loss": loss, "jaccard_index_value": jaccard_index_value}

    def validation_step(self, val_batch, batch_idx):
        image, mask = val_batch
        mask = mask.long()

        outputs = self.model(image)
        loss = self.criterion(outputs, mask)
        jaccard_index_value = jaccard_index(outputs.argmax(dim=1), mask, num_classes=2)

        self.log('val/loss', loss, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log('val/jaccard_index_value', jaccard_index_value, on_epoch=True, on_step=True, prog_bar=True,
                 sync_dist=True)

        return {"loss": loss, "jaccard_index_value": jaccard_index_value}

    def test_step(self, test_batch, batch_idx):
        image, mask = test_batch
        mask = mask.long()

        outputs = self.model(image)
        loss = self.criterion(outputs, mask)
        jaccard_index_value = jaccard_index(outputs.argmax(dim=1), mask, num_classes=2)

        self.log('test/loss', loss, on_epoch=True, on_step=False, prog_bar=True, sync_dist=True)
        self.log('test/jaccard_index_value', jaccard_index_value, on_epoch=True, on_step=False, prog_bar=True,
                 sync_dist=True)

        return {"loss": loss, "jaccard_index_value": jaccard_index_value}

In [ ]:
train_ds = SegmentationDataset(train, train_transform)
train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=12, shuffle=True, drop_last=True)

valid_ds = SegmentationDataset(valid, train_transform)
val_dataloader = torch.utils.data.DataLoader(valid_ds, batch_size=12)

In [ ]:
model = SegmentationModel()

In [ ]:
checkpoint_callback = ModelCheckpoint(
            monitor="val/jaccard_index_value",
            dirpath="checkpoints",
            filename="{val/jaccard_index_value:.4f}",
            save_top_k=3,
            mode="max",
            # save_weights_only=True
        )

early_stop_callback = EarlyStopping(monitor="val/loss", min_delta=0.00, patience=50, verbose=True,
                                            mode="min")

In [ ]:
trainer = pl.Trainer(accelerator='cuda',
                      devices=1,
                      max_epochs=10,
                      log_every_n_steps=1,
                      callbacks=[checkpoint_callback, early_stop_callback])

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [30]:
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | model     | UnetPlusPlus     | 6.6 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
6.6 M     Trainable params
0         Non-trainable params
6.6 M     Total params
26.279    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric val/loss improved. New best score: 0.117


...

INFO:pytorch_lightning.callbacks.early_stopping:Metric val/loss improved by 0.001 >= min_delta = 0.0. New best score: 0.078


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [31]:
trainer.logged_metrics

{'train/loss_step': tensor(0.0553),
 'train/jaccard_index_value_step': tensor(0.8480),
 'val/loss_step': tensor(0.0882),
 'val/jaccard_index_value_step': tensor(0.7189),
 'val/loss_epoch': tensor(0.0811),
 'val/jaccard_index_value_epoch': tensor(0.7950),
 'train/loss_epoch': tensor(0.0551),
 'train/jaccard_index_value_epoch': tensor(0.8325)}